In [ ]:
import numpy as np
import pandas as pd

In [ ]:
asdddddddddddddddddddddddddddddddddddddd

In [ ]:
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")

data = pd.concat([train, test], sort=False)

data['Sex'].replace(['male','female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['Age'].fillna(data['Age'].median(),inplace=True)
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

In [ ]:
data.head()

In [ ]:
delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis = 1)
X_test = test.drop('Survived', axis = 1)

In [ ]:
X_train.head()

## 機械学習アルゴリズム

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty='l2', solver="sag", random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0)

In [ ]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
y_pred[:10]

In [ ]:
y_pred = (y_pred > 0.5).astype(int)


In [ ]:
data.head()

In [ ]:
data['FamilySize'] = data['Parch'] + data['SibSp']+ 1
train['FamilySize'] = data['FamilySize'][:len(train)]
test['FamilySize'] = data['FamilySize'][len(train):]

import seaborn as sns
sns.countplot(x='FamilySize' , data = train,hue = 'Survived')

In [ ]:
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

train['IsAlone'] = data['IsAlone'][:len(train)]
test['IsAlone'] = data['IsAlone'][len(train):]

In [ ]:
sub = gender_submission

In [ ]:
sub['Survived'] = list(map(int,y_pred))
sub.to_csv("submission_randomforest.csv", index=False)
sub.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train,y_train,test_size=0.3,random_state=0,stratify=y_train)

In [ ]:
categorical_features = ['Embarked','Pclass','Sex']

In [ ]:
params = {
    'objective':'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

In [ ]:
import lightgbm as lgb


lgb_train = lgb.Dataset(X_train,y_train,categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid,reference=lgb_train,categorical_feature=categorical_features)


model = lgb.train(
    params,lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=10,
    num_boost_round=1000,
    early_stopping_rounds=10
)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
sub['Survived'] = y_pred
sub.to_csv("submission_lightgbm.csv", index=False)

sub.head()

In [ ]:
clf.fit(X_train, y_train)
y_pred_familysize_isalone = clf.predict(X_test)

sub['Survived'] = list(map(int,y_pred_familysize_isalone))
sub.to_csv("submission_familysize_isalone.csv", index = False)

sub.head()

In [ ]:
clf.fit(X_train.drop('FamilySize', axis = 1), y_train)
y_pred_isalone = clf.predict(X_test.drop('FamilySize', axis = 1))

sub['Survived'] = list(map(int, y_pred_isalone))
sub.to_csv("submission_isalone.csv" , index = False)

sub.head()

In [ ]:
clf.fit(X_train.drop('IsAlone', axis = 1), y_train)
y_pred_familysize = clf.predict(X_test.drop('IsAlone', axis = 1))

sub['Survived'] = list(map(int, y_pred_familysize))
sub.to_csv("submission_familysize.csv" , index = False)

sub.head()

In [ ]:
clf.fit(X_train.drop(['FamilySize','IsAlone'], axis = 1), y_train)
y_pred= clf.predict(X_test.drop(['FamilySize','IsAlone'], axis = 1))

sub['Survived'] = list(map(int, y_pred))
sub.to_csv("submission.csv" , index = False)

sub.head()

In [ ]:
import optuna
from sklearn.metrics import log_loss


def objective(trial):
    params = {
        'objective':'binary',
        'max_bin': trial.suggest_int('max_bin',255,500),
        'learning_rate': 0.05,
        'num_leaves': trial.suggest_int('num_leaves',32,128),
    }
    
    lgb_train = lgb.Dataset(X_train,y_train,categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_valid,y_valid,reference=lgb_train,categorical_feature=categorical_features)
    
    model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train,lgb_eval],
    verbose_eval=10,
    num_boost_round=1000,
    early_stopping_rounds=10
    )
    
    
    y_pred_valid = model.predict(X_valid,num_iteration=model.best_iteration)
    score = log_loss(y_valid,y_pred_valid)
    return score

In [ ]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective,n_trials=40)

In [ ]:
study.best_params

In [ ]:
from sklearn.model_selection import KFold


y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

categorical_features = ['Embarked', 'Pclass', 'Sex']

params = {
    'objective': 'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr, categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train, categorical_feature=categorical_features)

    model = lgb.train(
        params, lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        verbose_eval=10,
        num_boost_round=1000,
        early_stopping_rounds=10
    )

    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)

In [ ]:
pd.DataFrame(oof_train).to_csv('oof_train_kfold.csv', index=False)

scores = [
    m.best_score['valid_1']['binary_logloss'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

In [ ]:
from sklearn.metrics import accuracy_score


y_pred_oof = (oof_train > 0.5).astype(int)
accuracy_score(y_train, y_pred_oof)

In [ ]:
len(y_preds)

In [ ]:
y_preds[0][:10]

In [ ]:
y_sub = sum(y_preds) / len(y_preds)
y_sub = (y_sub > 0.5).astype(int)
y_sub[:10]

In [ ]:
sub['Survived'] = y_sub
sub.to_csv("submission_lightgbm_kfold.csv", index=False)

sub.head()

In [ ]:
from sklearn.model_selection import KFold


cv = KFold(n_splits=5, shuffle=True, random_state=0)
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    print(f'fold: {fold_id}')
    print(f'y_tr y==1 rate: {sum(y_tr)/len(y_tr)}')
    print(f'y_val y==1 rate: {sum(y_val)/len(y_val)}')

In [ ]:
from sklearn.model_selection import StratifiedKFold


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    print(f'fold: {fold_id}')
    print(f'y_tr y==1 rate: {sum(y_tr)/len(y_tr)}')
    print(f'y_val y==1 rate: {sum(y_val)/len(y_val)}')

In [ ]:
from sklearn.model_selection import StratifiedKFold


y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

categorical_features = ['Embarked', 'Pclass', 'Sex']

params = {
    'objective': 'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr, categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train, categorical_feature=categorical_features)

    model = lgb.train(
        params, lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        verbose_eval=10,
        num_boost_round=1000,
        early_stopping_rounds=10
    )

    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)

In [ ]:
pd.DataFrame(oof_train).to_csv('oof_train_skfold.csv', index=False)
print(oof_train[:10])

scores = [
    m.best_score['valid_1']['binary_logloss'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

In [ ]:
from sklearn.metrics import accuracy_score


y_pred_oof = (oof_train > 0.5).astype(int)
accuracy_score(y_train, y_pred_oof)

In [ ]:
y_sub = sum(y_preds) / len(y_preds)
y_sub = (y_sub > 0.5).astype(int)
y_sub[:10]

## 提出

In [ ]:
sub = pd.DataFrame(pd.read_csv('../input/titanic/test.csv')['PassengerId'])
sub['Survived'] = list(map(int, y_pred))
sub.to_csv("submission.csv", index=False)